# Implementation of Basic file operation in python

1. Problem Statement / Scenario

Problem: You are running a small local shop and need a simple system to manage your product inventory. You want to store product information (ID, Name, Category, Quantity, Price) in a file. Your system should allow you to:

* Set up an initial inventory.
* View all products in your inventory.
* Add new products.
* Update the quantity or price of existing products.
* Remove products from the inventory.

Solution: We will use a CSV (Comma Separated Values) file to store the inventory data, as it's human-readable and easy to work with using Python's built-in csv module.

# 1. Setup the initial configurations

In [17]:
import csv
import os # To handle file paths and check file existence

# --- Configuration ---
INVENTORY_FILE = "shop_inventory.csv"
HEADERS = ["ProductID", "ProductName", "Category", "Quantity", "Price"]

# 2. Define the helper functions to read and write data to/from csv

In [18]:
# --- Helper Function: Read all data from CSV ---

def _read_all_data(filepath):
    """Reads all rows from a CSV file, including the header."""
    data = []
    try:
        with open(filepath, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                data.append(row)
    except FileNotFoundError:
        print(f"File '{filepath}' not found. Creating a new one if writing.")
    except Exception as e:
        print(f"Error reading file '{filepath}': {e}")
    return data

# --- Helper Function: Write all data to CSV (overwrites existing) ---
def _write_all_data(filepath, data):
    """Writes all provided data (list of lists) to a CSV file, overwriting existing content."""
    try:
        with open(filepath, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerows(data)
        return True
    except IOError as e:
        print(f"Error writing to file '{filepath}': {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred while writing: {e}")
        return False

# 3. Define the function for  Initial Inventory / Setup

In [19]:
# --- Core Operations ---

# Operation 1: Create Initial Inventory / Setup (also used for saving updates/deletes)
def setup_initial_inventory():
    """
    Creates the initial CSV file with headers and some sample products
    if the file does not exist.
    """
    if not os.path.exists(INVENTORY_FILE):
        print(f"Creating initial inventory file: {INVENTORY_FILE}")
        initial_products = [
            HEADERS, # First row is always the header
            ["P001", "Laptop", "Electronics", 15, 1200.00],
            ["P002", "Keyboard", "Electronics", 50, 75.50],
            ["P003", "Notebook", "Stationery", 100, 5.25],
            ["P004", "Pen Set", "Stationery", 200, 12.00]
        ]
        _write_all_data(INVENTORY_FILE, initial_products)
    else:
        print(f"Inventory file '{INVENTORY_FILE}' already exists.")

# 4. Define the function for View Inventory

In [20]:
# Operation 2: View Inventory (Read)

def view_inventory():
    """Reads and displays the current inventory from the CSV file."""
    print("\n--- Current Inventory ---")
    data = _read_all_data(INVENTORY_FILE)

    if not data:
        print("Inventory is empty.")
        return

    # Print header
    print(f"{data[0][0]:<10} {data[0][1]:<20} {data[0][2]:<15} {data[0][3]:<10} {data[0][4]:<10}")
    print("-" * 70)

    # Print products
    for row_index, row in enumerate(data):
        if row_index == 0: # Skip header
            continue
        try:
            # Format output neatly
            product_id, name, category, quantity, price = row
            print(f"{product_id:<10} {name:<20} {category:<15} {int(quantity):<10} ${float(price):<9.2f}")
        except ValueError as e:
            print(f"  Error parsing row: {row}. Details: {e}")
        except IndexError:
            print(f"  Skipping malformed row: {row}")
    print("-------------------------\n")

# 5. Define the function for Add New Product

In [21]:
# Operation 3: Add New Product (Append)

def add_product(product_id, name, category, quantity, price):
    """Appends a new product to the inventory CSV."""
    new_product_row = [product_id, name, category, quantity, price]
    try:
        # Open in append mode ('a') to add to the end
        with open(INVENTORY_FILE, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(new_product_row)
        print(f"Added product: {name} ({product_id})")
    except IOError as e:
        print(f"Error adding product: {e}")

# 6. Define the function for Update Product

In [22]:
# Operation 4: Update Product (Read -> Modify in Memory -> Write)

def update_product(product_id, new_quantity=None, new_price=None):
    """
    Updates the quantity or price of an existing product by ProductID.
    Reads all data, modifies in memory, then rewrites the entire file.
    """
    data = _read_all_data(INVENTORY_FILE)
    if not data:
        print("Inventory is empty. Cannot update.")
        return

    updated = False
    updated_data = [data[0]] # Start with the header
    for row_index, row in enumerate(data):
        if row_index == 0: # Skip header in this loop, it's already added to updated_data
            continue

        if row[0] == product_id: # ProductID is the first element (index 0)
            print(f"Found product {product_id}. Original: Quantity={row[3]}, Price=${row[4]}")
            if new_quantity is not None:
                row[3] = new_quantity # Update Quantity (index 3)
            if new_price is not None:
                row[4] = new_price   # Update Price (index 4)
            updated = True
            print(f"Updated product {product_id}. New: Quantity={row[3]}, Price=${row[4]}")
        updated_data.append(row) # Add original or modified row to the new list

    if updated:
        if _write_all_data(INVENTORY_FILE, updated_data):
            print(f"Product {product_id} updated successfully in file.")
        else:
            print(f"Failed to save updates for product {product_id}.")
    else:
        print(f"Product with ID '{product_id}' not found.")

# 7. Define the function for Delete Product

In [23]:
# Operation 5: Delete Product (Read -> Filter in Memory -> Write)

def delete_product(product_id):
    """
    Deletes a product by ProductID.
    Reads all data, filters out the specified product in memory,
    then rewrites the entire file.
    """
    data = _read_all_data(INVENTORY_FILE)
    if not data:
        print("Inventory is empty. Nothing to delete.")
        return

    original_row_count = len(data)
    # Filter out the row to be deleted
    # Keep header and all rows where product ID does NOT match
    filtered_data = [data[0]] + [row for row in data[1:] if row[0] != product_id]

    if len(filtered_data) < original_row_count:
        if _write_all_data(INVENTORY_FILE, filtered_data):
            print(f"Product with ID '{product_id}' deleted successfully.")
        else:
            print(f"Failed to delete product {product_id}.")
    else:
        print(f"Product with ID '{product_id}' not found.")

# 8. Execute the operations

In [24]:
# --- Main Program Flow / Demonstrate Operations ---
if __name__ == "__main__":
    setup_initial_inventory() # Ensure the CSV file exists with initial data

    view_inventory()

    print("\n--- Adding a new product ---")
    add_product("P005", "Mouse Pad", "Accessories", 75, 8.99)
    view_inventory() # Show updated inventory

    print("\n--- Updating product P002 (Keyboard) quantity and price ---")
    update_product("P002", new_quantity=60, new_price=70.00)
    view_inventory() # Show updated inventory

    print("\n--- Attempting to update a non-existent product ---")
    update_product("P999", new_quantity=10)

    print("\n--- Deleting product P003 (Notebook) ---")
    delete_product("P003")
    view_inventory() # Show updated inventory

    print("\n--- Attempting to delete a non-existent product ---")
    delete_product("P888")

    print("\n--- Final Inventory ---")
    view_inventory()

Creating initial inventory file: shop_inventory.csv

--- Current Inventory ---
ProductID  ProductName          Category        Quantity   Price     
----------------------------------------------------------------------
P001       Laptop               Electronics     15         $1200.00  
P002       Keyboard             Electronics     50         $75.50    
P003       Notebook             Stationery      100        $5.25     
P004       Pen Set              Stationery      200        $12.00    
-------------------------


--- Adding a new product ---
Added product: Mouse Pad (P005)

--- Current Inventory ---
ProductID  ProductName          Category        Quantity   Price     
----------------------------------------------------------------------
P001       Laptop               Electronics     15         $1200.00  
P002       Keyboard             Electronics     50         $75.50    
P003       Notebook             Stationery      100        $5.25     
P004       Pen Set              S

# 9. Leverage pandas to visualize the data in tabular format as a dataframe

In [25]:
import pandas as pd

inventory = pd.read_csv('shop_inventory.csv')
inventory

,ProductID,ProductName,Category,Quantity,Price
0,P001,Laptop,Electronics,15,1200.00
1,P002,Keyboard,Electronics,60,70.00
2,P004,Pen Set,Stationery,200,12.00
3,P005,Mouse Pad,Accessories,75,8.99


# 10. Clean up the csv file after demonstration

In [26]:
# --- Optional: Clean up the created CSV file after demonstration ---

if os.path.exists(INVENTORY_FILE):
         os.remove(INVENTORY_FILE)
         print(f"\nCleaned up: '{INVENTORY_FILE}' removed.")


Cleaned up: 'shop_inventory.csv' removed.


# Key Insights

We can implement the above initial inventory problem very effectively using Pandas.

Pandas is more efficient for solving the inventory problem primarily due to its optimized performance for tabular data manipulation and significantly improved developer productivity.

* Optimized Performance: Pandas operations are implemented largely in C or highly optimized C extensions (like NumPy), meaning tasks like reading, filtering, updating, and writing data run much faster for even moderately sized datasets than equivalent operations written with pure Python loops and the csv module.
* Readability and Conciseness (Developer Efficiency): It provides a highly intuitive and concise syntax (e.g., df.loc[mask, 'column'] = value for updates, df[df['ID'] != id] for deletions, pd.read_csv() for reading) that drastically reduces the lines of code and complexity, leading to faster development and fewer bugs.
* Structured Data Handling: The DataFrame object is purpose-built for tabular data, automatically handling headers, data types, and providing powerful, built-in methods for common data tasks like sorting, grouping, and aggregation, which would require extensive manual coding with basic file operations.

# COMPLETED